### Student Name: Anant Singh || Student Number: 100797742

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

#### Scraping Best Selling Books on Amazon

In [2]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

#### Saving dataframe as a CSV file

In [3]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

#### Reading CSV File

In [4]:
df = pd.read_csv("amazon_products.csv")
df.shape
(100, 5)

(100, 5)

#### Veiwing the table

In [5]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,The Psychology of Money,Morgan Housel,4.5 out of 5 stars,"6,615",₹ 290.00
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"10,904",₹ 300.00
2,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5 out of 5 stars,"4,097",₹ 299.00
3,Grandma's Bag of Stories: Collection of 20+ Il...,Sudha Murty,4.6 out of 5 stars,"3,931",₹ 191.00
4,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"18,053",₹ 449.00
...,...,...,...,...,...
56,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.5 out of 5 stars,"43,672",₹ 292.00
57,Combo Pack: Lakhmir Singh Class 10 Science (Bi...,Manjit Kaur Lakhmir Singh,4.4 out of 5 stars,352,"₹ 1,180.00"
58,Oswaal CBSE Question Bank Class 12 Physics Cha...,Oswaal Editorial Board,4.6 out of 5 stars,420,₹ 365.00
59,Inner Engineering: A Yogi’s Guide to Joy,Sadhguru,4.6 out of 5 stars,"8,301",₹ 178.00


#### Preprocessing the ratings, customers_rated and price column

In [6]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
df['Rating'] = pd.to_numeric(df['Rating'])
df["Price"] = df["Price"].str.replace('₹', '')
df["Price"] = df["Price"].str.replace(',', '')
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])
df['Price'] = df['Price'].astype(int)
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,The Psychology of Money,Morgan Housel,4.5,6615,290
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,10904,300
2,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5,4097,299
3,Grandma's Bag of Stories: Collection of 20+ Il...,Sudha Murty,4.6,3931,191
4,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,18053,449


#### Verifying the data types of the DataFrame

In [7]:
df.dtypes
Book_Name:object
Author: object
Rating: float
Customers_Rated: int
Price: int
dtype: object

#### Replace the zero values in the DataFrame to NaN.

In [8]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

#### Counting the Number of NaNs in the DataFrame

In [9]:
count_nan = len(df) - df.count()
count_nan
Book_Name: 0
Author    :         6
Rating     :        0
Customers_Rated :   0
Price            :  1
dtype: int

#### Let's drop these NaNs.

In [10]:
df = df.dropna()

#### Authors Highest Priced Book

In [11]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:10]
data

,Book Name,Author,Rating,Customers_Rated,Price
92,Essentials of Medical Pharmacology,K. D. Tripathi,4.6,1199.0,1395
57,Combo Pack: Lakhmir Singh Class 10 Science (Bi...,Manjit Kaur Lakhmir Singh,4.4,352.0,1180
83,New Simplified Physics : A Reference Book for ...,S.L. Arora,4.3,1193.0,1050
89,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,3338.0,799
15,Atomic Habits: The life-changing million copy ...,James Clear,4.6,15817.0,600
7,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,6462.0,595
28,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.7,830.0,550
9,Mathematics for Class 10 by R D Sharma (Examin...,R.D. Sharma,4.5,902.0,545
85,How to Prepare for Quantitative Aptitude for t...,Arun Sharma,4.5,2111.0,545
29,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,4030.0,540


#### Visualize the results for such top 20 authors

In [12]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

Loading BokehJS ...

In [13]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)

#### Top Rated Books and Authors wrt Customers Rated

In [14]:
data = df[df['Customers_Rated'] > 1000]
data = data.sort_values(['Rating'],axis=0, ascending=False)[:10]
data

,Book Name,Author,Rating,Customers_Rated,Price
84,Diary of a Wimpy Kid: The Deep End (Book 15),Jeff Kinney,4.7,4690.0,379
16,Think Like a Monk: The secret of how to harnes...,Jay Shetty,4.7,10689.0,275
65,Death; An Inside Story: A book for all those w...,Sadhguru,4.7,3732.0,202
68,"The Magic of the Lost Temple: Illustrated, eas...",Sudha Murty,4.7,1880.0,199
20,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,30170.0,340
43,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,24455.0,288
95,The Theory of Everything: The Origin and Fate ...,Stephen Hawking,4.6,4498.0,169
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,10904.0,300
52,Indian Art and Culture for Civil Services and ...,Nitin Singhania,4.6,1904.0,493
31,Oxford Student Atlas for India - Third Edition,Oxford University Press,4.6,3929.0,210


In [15]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)

In [16]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)


#### Most Customer Rated Authors and Books

In [17]:
data = df.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]
data

,Book Name,Author,Rating,Customers_Rated,Price
97,"The Silent Patient: The record-breaking, multi...",Alex Michaelides,4.5,62631.0,288
5,The Alchemist,Paulo Coelho,4.6,47431.0,172
56,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.5,43672.0,292
30,Think and Grow Rich,Napoleon Hill,4.5,42584.0,149
81,How to Win Friends and Influence People,Dale Carnegie,4.5,40071.0,159
11,Rich Dad Poor Dad: What the Rich Teach Their K...,Robert T. Kiyosaki,4.6,36963.0,382
20,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,30170.0,340
35,Man's Search For Meaning: The classic tribute ...,Viktor E Frankl,4.5,27902.0,180
25,The Power of Your Subconscious Mind,Joseph Murphy,4.5,27140.0,88
43,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,24455.0,288


In [18]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data["Author"])
p = figure(plot_width=700, plot_height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data,fill_alpha=0.6, fill_color=index_cmap,size=20,legend='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'
BokehDeprecationWarning: "'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead"
show(p)
